# Data pre-processing & tokenization

CS685 Spring 2022 <br />
Feb. 24, 2022<br />
Hongyu Tu <br />

In [6]:
import re
import time
import jieba
import pickle
import asyncio
import datetime
import requests
import numpy as np
import pandas as pd 
import nest_asyncio
from utils import *
import jieba.analyse
from tqdm import tqdm
from os import listdir
from collections import Counter
from bilibili_api import video, sync

jieba.initialize() 
nest_asyncio.apply()
# jieba.analyse.set_stop_words('stopwords.txt')

In [7]:
def pre_p(name):
    df = pd.read_csv('{}.csv'.format(name))

    f = open('stopwords.txt', encoding="utf8")
    stop_word = f.read().split('\n')
    f.close()
    
    cat_dic = init_category_dic()
    tid_lst = np.array(list(cat_dic.keys()))
    
    text_lst = df['danmu' if 'danmu' in name else 'comment'].tolist()
    fq_lst = df['Frequency'].tolist()
    ct_lst = df['Category ID'].tolist()
    
    token_dict = {}
    for idx in tqdm(range(len(text_lst))):
        tid = ct_lst[idx]
        if not pd.isna(text_lst[idx]) and tid in tid_lst: 
            lst_idx = np.where(tid_lst==tid)[0][0]

            seg_list = list(jieba.cut_for_search(text_lst[idx]))
            if len(text_lst[idx]) <= 6:
                seg_list.append(text_lst[idx])
            for token in seg_list:
                # token not in stop_word and 
                if len(token) > 2 and len(set([_ for _ in token])) > 1:
                    if token in token_dict:
                        tmp = np.eye(126)[lst_idx] * fq_lst[idx]
                        token_dict[token] = token_dict[token] + tmp
                    else:
                        token_dict[token] = np.eye(126)[lst_idx] * fq_lst[idx]
                        
    token_lst = np.array(list(token_dict.keys()))
    cnt_lst = np.array([sum(token_dict[i]) for i in token_lst])
    dist_lst = np.array(list(token_dict.values()))/cnt_lst[:,None]
    general_lst = np.array([len(np.where(i != 0)[0]) for i in dist_lst])
    
    df2 = pd.DataFrame(np.array([token_lst, cnt_lst, general_lst, ]).T, \
                   columns = ['Word Token', 'Total Apperance', 'Category Apperance'])
    return df2

In [8]:
danmu_df, comment_df = pre_p('danmu'), pre_p('comment')

100%|████████████████████████████████████████████████████████████████████████| 127002/127002 [00:19<00:00, 6522.26it/s]


In [16]:
danmu_df.sort_values(by=['Category Apperance'])[-15:]

,Word Token,Total Apperance,Category Apperance
77995,三年级,32.0,9
62802,给爷火,17.0,9
100632,耶格尔,36.0,9
7585,一个头,15.0,9
11175,半条命,9.0,9
11197,取关了,10.0,9
32183,一天到晚,11.0,9
4209,20分钟,12.0,9
60068,高攀不起,12.0,9
24853,456,20.0,9


In [15]:
danmu_df.sort_values(by=['Total Apperance'])[-15:]

,Word Token,Total Apperance,Category Apperance
15573,我哭了,98.0,18
15903,看哭了,98.0,24
334191,已三连求资料,99.0,2
63458,好好看啊,99.0,27
51943,王侯将相,99.0,4
12380,三分钟,99.0,40
18190,蜘蛛侠,99.0,24
29797,cao,99.0,26
27451,律师函,99.0,24
3539,憋不住,99.0,33


In [14]:
comment_df.sort_values(by=['Category Apperance'])[-15:]

,Word Token,Total Apperance,Category Apperance
2786,总有一天,16.0,9
4724,对得起,35.0,9
5188,发动机,294.0,9
7665,公务员,24.0,9
261,非常感谢,29.0,9
787,消费者,180.0,9
1981,完整版,18.0,9
593,实话实说,15.0,9
595,用不上,22.0,9
7968,深有体会,14.0,9


In [13]:
comment_df.sort_values(by=['Total Apperance'])[-15:]

,Word Token,Total Apperance,Category Apperance
2791,下雪天,94.0,2
3226,2008,94.0,3
682,一模一样,94.0,19
312,所有人,94.0,17
85,嗑瓜子,948.0,24
47981,三连求资料,95.0,3
11776,防滑链,97.0,1
456,看不懂,97.0,18
93,翻白眼,97.0,16
2122,看不到,98.0,18
